In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import torchvision
from fastai.text import *


ModuleNotFoundError: No module named 'bcolz'

In [3]:
import sys
print(sys.path)

['', '/home/ray/anaconda3/envs/fastai-please-work/lib/python36.zip', '/home/ray/anaconda3/envs/fastai-please-work/lib/python3.6', '/home/ray/anaconda3/envs/fastai-please-work/lib/python3.6/lib-dynload', '/home/ray/anaconda3/envs/fastai-please-work/lib/python3.6/site-packages', '/home/ray/anaconda3/envs/fastai-please-work/lib/python3.6/site-packages/IPython/extensions', '/home/ray/.ipython']


In [4]:
import sys
sys.path.insert(0, '/home/ray/anaconda3/envs/fastai/lib/python3.6/site-packages/')
import fastai.vision

ModuleNotFoundError: No module named 'fastai.vision'

In [5]:
import fastai.utils.collect_env; fastai.utils.collect_env.show_install(1)

ModuleNotFoundError: No module named 'fastai.utils'

In [ ]:
from fastai.utils.show_install import *
show_install()

In [ ]:
from fastai.vision import *
from fastai.callbacks.hooks import *
torch.cuda.set_device(0)

In [ ]:
torch.cuda.is_available()


In [ ]:
torch.backends.cudnn.enabled


In [ ]:
path = untar_data(URLs.CAMVID)
path.ls()

In [ ]:
path_lbl = path/'labels'
path_img = path/'images'

In [ ]:
fnames = get_image_files(path_img)
fnames[:3]